In [1]:
import cdsapi
import xarray as xr
import time
import rioxarray as rio

In [2]:
import sys
sys.path.append('..')

In [3]:
import utils

In [ ]:
start_time = time.time()

dataset = "derived-era5-single-levels-daily-statistics"
request = {
    "product_type": "reanalysis",
    "variable": ["total_precipitation",],
    "year": "2020",
    "month": ["01"],
    "day": ["01"],
    "daily_statistic": "daily_sum",
    "time_zone": "utc+00:00",
    "frequency": "1_hourly",
    "format": "netcdf",
}

client = cdsapi.Client()
client.retrieve(dataset, request).download("../data/output.nc")

end_time = time.time()

print(f't_elapsed: {end_time - start_time}')

In [43]:
ncfile = xr.open_dataset('../data/output.nc')

In [ ]:
ncfile

In [ ]:
ncfile['tp'].coords['longitude']

In [70]:
tp = ncfile['tp']
tp = tp * 1000 # m to mm
tp.coords['longitude'] = (tp.coords['longitude'] + 180) % 360 - 180
tp = tp.sortby('longitude')

In [72]:
tp = tp.rio.set_spatial_dims('longitude', 'latitude')
tp = tp.rio.write_crs('epsg:4326')

In [75]:
tp.rio.to_raster('../data/output.tif', compress='zstd')

In [ ]:
import datetime

utils.download_data(
    variable = '2m_temperature',
    daily_statistic = 'daily_mean',
    date = datetime.datetime(2020, 2, 1),
    download_filepath = '../data/2mtemp_2021_02.nc',
    exist_ok = True,
    request_full_month = True,
)

In [ ]:
12 * 30 * 172 / 3600

In [ ]:
ncfile = xr.open_dataset('../data/2mtemp_2021_02.nc')
ncfile

In [12]:
t2m = ncfile['t2m']
t2m = t2m - 273.15 # K to deg C
t2m.coords['longitude'] = (t2m.coords['longitude'] + 180) % 360 - 180
t2m = t2m.sortby('longitude')
t2m = t2m.rio.set_spatial_dims('longitude', 'latitude')
t2m = t2m.rio.write_crs('epsg:4326')
t2m.rio.to_raster('../data/2mtemp_2020_02.tif', compress='zstd')

In [ ]:
t2m

In [11]:
import geopandas as gpd

malawi_gdf = gpd.read_file('/Users/nikhilsrajan/NASA-Harvest/project/fetch_satdata/data/malawi/mwi_adm_nso_hotosm_20230405_shp/mwi_admbnda_adm0_nso_hotosm_20230405.shp')

In [ ]:
def load_mean_temperature_nc_file(nc_filepath:str):
    ncfile = xr.open_dataset(nc_filepath)
    t2m = ncfile['t2m']
    t2m = t2m - 273.15 # K to deg C
    t2m.coords['longitude'] = (t2m.coords['longitude'] + 180) % 360 - 180
    t2m = t2m.sortby('longitude')
    t2m = t2m.rio.set_spatial_dims('longitude', 'latitude')
    t2m = t2m.rio.write_crs('epsg:4326')
    return t2m


def load_total_precipitation_nc_file(nc_filepath:str):
    ncfile = xr.open_dataset(nc_filepath)
    tp = ncfile['tp']
    tp = tp * 1000 # m to mm
    tp.coords['longitude'] = (tp.coords['longitude'] + 180) % 360 - 180
    tp = tp.sortby('longitude')
    tp = tp.rio.set_spatial_dims('longitude', 'latitude')
    tp = tp.rio.write_crs('epsg:4326')
    return tp

In [ ]:
150 * (4930 * 2) / (64 / 16) / 3600 / 24

In [ ]:
45 * (4930 * 2) / (64 / 16) / 3600

In [ ]:
import json

with open('../data/cdsapi_credentials.json', 'r') as h:
    credentials = json.load(h)

URL = credentials['url']
KEY = credentials['key']

URL, KEY